API für DinoProject

Methoden:

POST /game: Erstellt eine neue Session und gibt einen Redirect zurück
 
GET /highscore : Liefert alle Session-IDs aus der DB zurück <br/>
POST /highscore : Erstellt eine neue Session in der DB

GET /highscore/{id} : Liefert zu einer Session-ID den Highscore zurück <br/>
PUT /highscore/{id} : Aktualisiert bei einer Session den Highscore

In [1]:
from flask import Flask, request
from flask_restful import Resource, Api
from flask_cors import CORS
from sqlalchemy import create_engine
from json import dumps
from flask import jsonify, redirect
import optparse
import mysql.connector
import uuid

app = Flask(__name__)
cors = CORS(app, resources={r"*": {"origins": "*"}}) # CORS
api = Api(app)


# Datenbank konfigurieren und Verbindung herstellen
mydb = mysql.connector.connect(
    host = "webengineering.ins.hs-anhalt.de",
    port = 32192,
    user = "root",
    passwd = "ga4wm19",
    database = "dinoProject"
)

mycursor = mydb.cursor()
mydb.disconnect() # Vorläufig Verbindung schließen

In [2]:
# Klasse um eine neue Session zu erstellen
class createSession(Resource):
    def post(self):
        sessionID = str(uuid.uuid4().hex)
        return redirect('http://localhost:8080/game/' + sessionID, code=303)
    
# Klasse für /highscore
class createNewHighscore(Resource):
    
    # Alle Highscores holen
    def get(self):
        result = []
        
        # Datenbank reconnect
        mydb.reconnect(attempts = 1, delay = 0)
        # SQL Query
        mycursor.execute("SELECT * FROM dinoProject.highscoreTable")
        # Rückgabe holen
        dbResult = mycursor.fetchall()
        
        # Jede Session an result anhängen
        for session in dbResult:
            result.append({'id': session[0], 'highscore': session[1], 'sessionID': session[2]})
        
        # Verbindung schließen
        mydb.disconnect()
        
        if len(result) > 0:
            return jsonify(result)
        else:
            return jsonify({'status': 'no existing highscores', 'statuscode': 200})
        
    # Neuen Highscores erstellen
    def post(self):
        # Wenn übergebenes Objekt unvollständig
        if not request.json or not "highscore" in request.json or not "sessionID" in request.json:
            return jsonify({'status': 'could not add session', 'statuscode': 409})
        else:
            sql = 'INSERT INTO dinoProject.highscoreTable (highscore, sessionID) VALUES (%s, %s)'
            values = (request.json["highscore"], request.json["sessionID"])
            mydb.reconnect(attempts = 1, delay = 0)
            mycursor.execute(sql, values)
            mydb.commit() # Änderungen an DB commiten
            mydb.disconnect()
            
            return jsonify({'status': 'highscore added', 'statuscode': 200})

# Klasse für Updaten und Abfragen eines spezifischen Highscores
class getOrUpdateHighscore(Resource):
    def get(self, id):
        sql = "SELECT * FROM dinoProject.highscoreTable WHERE sessionID = '" + id + "'"
        mydb.reconnect(attempts = 1 , delay = 0)
        mycursor.execute(sql)
        dbResult = mycursor.fetchall()
        
        if len(dbResult) == 1 :
            session = dbResult[0]
            return jsonify({'id': session[0], 'highscore': session[1], 'sessionID': session[2]})
        else:
            return jsonify({'status': 'none or more than one highscore found', 'statuscode': 409})
        
    def put(self, id):
        if not request.json or not "highscore" in request.json or not "sessionID" in request.json:
            return jsonify({'status': 'could not update highscore', 'statuscode': 409})
        else:
            sql = "UPDATE dinoProject.highscoreTable SET highscore = %s WHERE sessionID = %s"
            values = (request.json["highscore"], request.json["sessionID"])
            mydb.reconnect(attempts = 1 , delay = 0)
            mycursor.execute(sql, values)
            mydb.commit()
            mydb.disconnect()
            return jsonify({'status': 'updated highscore with id: ' + request.json["sessionID"], 'statuscode': 200})

In [3]:
# Klassen an entsprechende URLs binden
api.add_resource(createSession, '/game')
api.add_resource(createNewHighscore, '/highscore')
api.add_resource(getOrUpdateHighscore, '/highscore/<id>')

# Run für Testzwecke, startet auf localhost:5000
if __name__ == "__main__":
    app.run(port=4000)

# Run für Dockercontainer, mit Portangabe
#if __name__ == "__main__":
 #   parser = optparse.OptionParser(usage="python server.py -p ")
  #  parser.add_option('-p', '--port', action='store', dest='port', help='The port to listen on.')
   # (args, _) = parser.parse_args()
    #if args.port == None:
     #   print "Missing required argument: -p/--port"
      #  sys.exit(1)
    #app.run(host='0.0.0.0', port=int(args.port), debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Sep/2019 10:49:51] "POST /game HTTP/1.1" 303 -
127.0.0.1 - - [03/Sep/2019 10:53:59] "POST /game HTTP/1.1" 303 -
127.0.0.1 - - [03/Sep/2019 10:55:36] "POST /game HTTP/1.1" 303 -
